# Resolution in Python

Brainstorming:
- Descriptor
    * Rollback example
    * Proxy pattern (lazy)
- MRO: just mention, topic of another lecture
- `getattr`, `setattr`
- Monkey Patching and mocking
- Mention slot?

## Basic lookup

When accessing the attribute of an object, Python will proceed as follow:
1. invoke que the `__getattribute__` method, which is responsible for
2. looking into the instance `__dict__` for the attribute
3. looking into the class `__dict__` if the former fails
4. delegate to the `__getattr__` method for custom handling
5. if `__getattr__` is not implemented, an `AttributeError` is raised.

This lookup will be refined as we introduced new concepts.

> :warning: :skull: Note that if the class is defined with `slot`, the instance `__dict__` will be missing (unless it is explictly re-created as a slot).

Let's invsetigate the lookup step by step.

In [ ]:
# Attribute lookup
class Rectangle:

    def __init__(self, width, height):
        self.width = width
        self.height = height


    def __getattribute__(self, name):
        print(f"Accessing attribute: {name}")
        return super().__getattribute__(name)



r = Rectangle(10, 20)
print(r.width)
print(r.__dict__)

print("---")
r.width = 15
print(r.__dict__)

Accessing attribute: width
10
Accessing attribute: __dict__
{'width': 10, 'height': 20}
---
Accessing attribute: __dict__
{'width': 15, 'height': 20}


> :warning: :skull: Note that Python is not using the `__getitem__` method of the dictionary (at least in CPython) but does a more direct lookup

In the case of a class attribute, or method, we end up in the class `__dict__`

In [17]:
# Method and class attribute lookup
class Rectangle:

    _AUTHOR_ = "JMB"

    def __init__(self, width, height):
        self.width = width
        self.height = height


    def __getattribute__(self, name):
        print(f"Accessing attribute: {name}")
        return super().__getattribute__(name)

    def area(self):
        return self.width * self.height


r = Rectangle(10, 20)
print(r.__dict__)  # Only attributes

print("---")
r._AUTHOR_ # goes through the __getattribute__ method
r.area()   # goes through the __getattribute__ method

print("---")
print(Rectangle.__dict__)  # Contains _AUTHOR_ and area

Accessing attribute: __dict__
{'width': 10, 'height': 20}
---
Accessing attribute: _AUTHOR_
Accessing attribute: area
Accessing attribute: width
Accessing attribute: height
---
{'__module__': '__main__', '_AUTHOR_': 'JMB', '__init__': <function Rectangle.__init__ at 0x7f5d646691f0>, '__getattribute__': <function Rectangle.__getattribute__ at 0x7f5d64669550>, 'area': <function Rectangle.area at 0x7f5d64669040>, '__dict__': <attribute '__dict__' of 'Rectangle' objects>, '__weakref__': <attribute '__weakref__' of 'Rectangle' objects>, '__doc__': None}


Interestingly, when performing a lookup for a method, Python finds it in the **class** `__dict__` and then binds it to the instance. This is different from the following

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        def area():
            return self.width * self.height
        self.area = area

r = Rectangle(10, 20)
print(r.__dict__)  # Contains area as well


{'width': 10, 'height': 20, 'area': <function Rectangle.__init__.<locals>.area at 0x7f5d646690d0>}


## Inheritance

### Simple inheritance

What happens in case of inheritance?

In [22]:
class Rectangle:

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height
    

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)
    

print(Square.__dict__)  # area not in Square.__dict__
print(Square.area)  # Accessible even though it is not in Square.__dict__

{'__module__': '__main__', '__init__': <function Square.__init__ at 0x7f5d646698b0>, '__doc__': None}
<function Rectangle.area at 0x7f5d64669820>


### Multiple inheritance and MRO

How does the lookup proceed in case of complex inheritance?

Inheritance:
- illustrate that attribute from parent class are in the self dict
- illustrate MRO in the resolutio